# Agent : Learning

## Backtest

In [1]:

from system.agent import Agent
from system.env import Env, GEnv

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


START = "2022"
END = "2024"
TICKERS = ["BTC", "ETH", "SOL", "QNT", "DASH", "MATIC", "TWT", "GALA", "EGLD", "XMR"]

SYMBOL = "QNT"
INTERVAL = "1d"
CAPITAL = 1000


config = {
    "symbols" : TICKERS, "interval" : INTERVAL, "start" : START, "end" : END
    }
env = GEnv(**config)

sub_env = Env(symbol=SYMBOL, interval=INTERVAL, start=START, end=END)

# Agent details
CAPITAL = 1000
LEVERAGE = 5

n_SESSION = 100

AGENT_PARAMS = [
    [(1, "BTC"), CAPITAL, sub_env, n_SESSION],
    [(2, "BTC"), CAPITAL, sub_env, n_SESSION],
    [(3, "BTC"), CAPITAL, sub_env, n_SESSION],
    [(4, "BTC"), CAPITAL, sub_env, n_SESSION],
    [(5, "BTC"), CAPITAL, sub_env, n_SESSION]
]


STRATEGIES = [
    ("MOMENTUM", 3, {"floor" : 4}),
    ("TRIPLEMA", (3, 7, 14), {"floor" : 4}),
    ("MOMENTUM", 8, {"floor" : 4}),
    ("TRIPLEMA", (14, 21, 35), {"floor" : 4})
]

STRATEGIES = [
    ("TRIPLEMA", (3, 7, 14), {"floor" : 4})
]


STRATEGIES = [
    ("MOMENTUM", 3, {"floor" : 4}),
    ("MOMENTUM", 5, {"floor" : 4}),
    ("MOMENTUM", 6, {"floor" : 4}),
    ("MOMENTUM", 8, {"floor" : 4}),
    ("MOMENTUM", 9, {"floor" : 4})
]

AGENT = []
for agent_x, strategie in zip(AGENT_PARAMS, STRATEGIES):
    print(agent_x)
    agent = Agent(*agent_x)
    agent.update_policy(*strategie)
    agent.run_episode()
    AGENT.append(agent)
    


[(1, 'BTC'), 1000, <system.env.Env object at 0x0000025118245950>, 100]
{'date': Timestamp('2022-04-09 00:00:00'), 'nbTrades': 10, 'winRate': 0.3, 'lossRate': -0.7, 'amountWin': 133.4628879955868, 'amountLoss': -234.97453700023686, 'totalAmount': -101.51164900465005, 'amount': -92.84321435036986, 'expectancy': -10.151164900465002, 'profitFactor': 0.5679887263506015, 'minExposure': 2, 'maxExposure': 2, 'q1 avg median q3 %': [-7.0868, -0.9722, -7.039, -6.9913]}
__ End Session 0 ____________
__ Start Session 1____________
{'date': Timestamp('2022-07-19 00:00:00'), 'nbTrades': 21, 'winRate': 0.3333333333333333, 'lossRate': -0.6666666666666666, 'amountWin': 531.6554154213645, 'amountLoss': -413.5127165921142, 'totalAmount': 118.14269882925032, 'amount': 141.89556779282407, 'expectancy': 5.625842801392871, 'profitFactor': 1.2857051164058526, 'minExposure': 2, 'maxExposure': 2, 'q1 avg median q3 %': [-8.3204, 0.8891, -8.2985, -8.2766]}
__ End Session 1 ____________
__ Start Session 2__________

In [4]:
AGENT[0].view_report()

(1, 'BTC')


In [9]:
AGENT[0].session.rets_dist.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7])

In [5]:
AGENT[0].get_report()

Initial Capital: 1000
Agent Capital: 546.7600080393555
Portfolio Capital: 735.3285472530172
PnL: -453.2399919606445
Return :  -45.32399919606445 %
Min exposure : 2  Max exposure : 2


,date,nbTrades,winRate,lossRate,profitFactor
0,2022-04-09,10,0.300000,-0.700000,0.567989
1,2022-07-19,21,0.333333,-0.666667,1.285705
2,2022-10-28,21,0.285714,-0.714286,0.878575
3,2023-02-06,21,0.238095,-0.761905,0.906733
4,2023-05-18,20,0.400000,-0.600000,2.019551
5,2023-08-27,18,0.277778,-0.722222,1.648123
6,2023-12-06,20,0.400000,-0.600000,0.861350
7,2024-03-16,21,0.476190,-0.523810,2.154103
8,2022-04-09,9,0.444444,-0.555556,0.299524
9,2022-07-19,18,0.222222,-0.777778,0.704385


In [10]:
session = AGENT[0].post_event.sessionData
session

,date,nbTrades,winRate,lossRate,amountWin,amountLoss,totalAmount,amount,expectancy,profitFactor,minExposure,maxExposure,q1 avg median q3 %
0,2022-04-09,10,0.300000,-0.700000,133.462888,-234.974537,-101.511649,-92.843214,-10.151165,0.567989,2,2,"[-7.0868, -0.9722, -7.039, -6.9913]"
1,2022-07-19,21,0.333333,-0.666667,531.655415,-413.512717,118.142699,141.895568,5.625843,1.285705,2,2,"[-8.3204, 0.8891, -8.2985, -8.2766]"
2,2022-10-28,21,0.285714,-0.714286,315.762809,-359.403345,-43.640536,23.773913,-2.078121,0.878575,2,2,"[-6.8117, -0.1233, -6.811, -6.8104]"
3,2023-02-06,21,0.238095,-0.761905,338.081556,-372.856824,-34.775268,-54.813877,-1.655965,0.906733,2,2,"[-7.5152, 0.0134, -7.458, -7.4007]"
4,2023-05-18,20,0.400000,-0.600000,394.103067,-195.143894,198.959172,137.174418,9.947959,2.019551,2,2,"[-4.4503, 1.0477, -4.3893, -4.3283]"
5,2023-08-27,18,0.277778,-0.722222,278.201163,-168.798736,109.402427,246.576845,6.077913,1.648123,2,2,"[-3.565, 0.5979, -3.542, -3.5189]"
6,2023-12-06,20,0.400000,-0.600000,323.589640,-375.677162,-52.087522,380.915680,-2.604376,0.861350,2,2,"[-6.4853, -0.093, -6.4814, -6.4774]"
7,2024-03-16,21,0.476190,-0.523810,616.849748,-286.360403,330.489345,617.118736,15.737588,2.154103,2,2,"[-4.9553, 1.3058, -4.8885, -4.8218]"
8,2022-04-09,9,0.444444,-0.555556,86.586412,-289.079571,-202.493160,546.150442,-22.499240,0.299524,2,2,"[-11.2653, -2.3265, -11.2224, -11.1796]"
9,2022-07-19,18,0.222222,-0.777778,346.813704,-492.364169,-145.550465,455.630849,-8.086137,0.704385,2,2,"[-10.737, -0.739, -10.6307, -10.5244]"


In [19]:
from system.agent import Agent
from system.env import Env, GEnv

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


START = "2021"
END = "2024"
TICKERS = ["BTC", "ETH", "SOL", "QNT", "DASH", "MATIC", "TWT", "GALA", "EGLD", "XMR"]

SYMBOL = "QNT"
INTERVAL = "1d"
CAPITAL = 1000


config = {
    "symbols" : TICKERS, "interval" : INTERVAL, "start" : START, "end" : END
    }
env = GEnv(**config)

sub_env = Env(symbol=SYMBOL, interval=INTERVAL, start=START, end=END)

# Agent details
CAPITAL = 1000
LEVERAGE = 5

AGENT_PARAMS = [
    [(1, SYMBOL), CAPITAL, sub_env, n_SESSION]
]


STRATEGIES = [
    ("MOMENTUM", 6, {"floor" : 4})
]


AGENT = []
for agent_x, strategie in zip(AGENT_PARAMS, STRATEGIES):
    print(agent_x)
    agent = Agent(*agent_x)
    agent.update_policy(*strategie)
    agent.run_episode()
    AGENT.append(agent)
    


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[(1, 'QNT'), 1000, <system.env.Env object at 0x0000018A8B5E1650>, 100]
{'date': Timestamp('2021-11-04 00:00:00'), 'nbTrades': 10, 'winRate': 0.2, 'lossRate': -0.8, 'amountWin': 24.71109553056499, 'amountLoss': -505.94803382008763, 'totalAmount': -481.23693828952264, 'amount': -453.2107406801194, 'expectancy': -48.12369382895226, 'profitFactor': 0.04884117316157438, 'minExposure': 2, 'maxExposure': 2, 'q1 avg median q3 %': [-15.5045, -6.1736, -15.4816, -15.4588]}
__ End Session 0 ____________
__ Start Session 1____________
{'date': Timestamp('2022-02-11 00:00:00'), 'nbTrades': 14, 'winRate': 0.5714285714285714, 'lossRate': -0.42857142857142855, 'amountWin': 302.2994361903492, 'amountLoss': -182.03712607326668, 'totalAmount': 120.2623101170825, 'amount': -360.9746281724399, 'expectancy': 8.590165008363037, 'profitFactor': 1.6606471586937661, 'minExposure': 2, 'maxExposure': 2, 'q1 avg median q3 %': [-

In [20]:
AGENT[0].view_report()

(1, 'QNT')


In [ ]:
pp = AGENT[0].env.post_event.portfolioData
pp.head()

In [ ]:
dd = AGENT[0].env.post_event.tradeData
dd

In [ ]:
import ast

serie = dd["pnl_pct"].values
serie = dd[dd["state"].apply(lambda x : ast.literal_eval(x)[0]) == 'Close']["pnl_pct"].values

In [ ]:
import ast

dd["state"].apply(lambda x : ast.literal_eval(x)[1]).iloc[-1]

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Boxplots
fig_box = go.Figure()
fig_box.add_trace(go.Box(y=serie_pos, name='Positives', marker_color='blue'))
fig_box.add_trace(go.Box(y=serie_neg, name='Négatives', marker_color='red'))
fig_box.update_layout(title_text='Boxplots des valeurs positives et négatives', yaxis_title='Valeur')

fig_box.update_layout(height = 800 , width = 1000,
                          legend = dict(orientation="h",
                                        yanchor="bottom", y=1,
                                        xanchor="right", x=0.5),
                          margin = {'t':0, 'b':0, 'l':10, 'r':0}
                          )

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Exemple de série de données
data = serie  # Génère 1000 valeurs aléatoires selon une distribution normale

# Séparer les données en positives et négatives
positive_data = data[data > 0]
negative_data = data[data < 0]

# Afficher les histogrammes
plt.figure(figsize=(12, 6))

# Histogramme des valeurs positives
plt.subplot(1, 2, 1)
plt.hist(positive_data, bins=30, color='blue', alpha=0.7, edgecolor='black')
plt.title('Histogramme des valeurs positives')
plt.xlabel('Valeur')
plt.ylabel('Fréquence')

# Histogramme des valeurs négatives
plt.subplot(1, 2, 2)
plt.hist(negative_data, bins=30, color='red', alpha=0.7, edgecolor='black')
plt.title('Histogramme des valeurs négatives')
plt.xlabel('Valeur')
plt.ylabel('Fréquence')

plt.tight_layout()
plt.show()

# Afficher les KDE
plt.figure(figsize=(12, 6))

# KDE des valeurs positives
sns.kdeplot(positive_data, shade=True, color='blue', label='Positives')
# KDE des valeurs négatives
sns.kdeplot(negative_data, shade=True, color='red', label='Négatives')

plt.title('Estimation de la densité de probabilité')
plt.xlabel('Valeur')
plt.ylabel('Densité')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = serie

# Séparer les données en positives et négatives
positive_data = data[data > 0]
negative_data = data[data < 0]

# Création de la figure et des sous-graphiques
fig, axs = plt.subplots(2, 2, figsize=(14, 12))

# Histogramme et KDE des valeurs positives
sns.histplot(positive_data, bins=30, kde=True, color='blue', ax=axs[0, 0])
axs[0, 0].set_title('Histogramme et KDE des valeurs positives')
axs[0, 0].set_xlabel('Valeur')
axs[0, 0].set_ylabel('Fréquence/Densité')

# Histogramme et KDE des valeurs négatives
sns.histplot(negative_data, bins=30, kde=True, color='red', ax=axs[0, 1])
axs[0, 1].set_title('Histogramme et KDE des valeurs négatives')
axs[0, 1].set_xlabel('Valeur')
axs[0, 1].set_ylabel('Fréquence/Densité')

# Boxplot des valeurs positives
sns.boxplot(data=positive_data, ax=axs[1, 0], color='blue')
axs[1, 0].set_title('Boxplot des valeurs positives')
axs[1, 0].set_xlabel('Valeur')

# Boxplot des valeurs négatives
sns.boxplot(data=negative_data, ax=axs[1, 1], color='red')
axs[1, 1].set_title('Boxplot des valeurs négatives')
axs[1, 1].set_xlabel('Valeur')

plt.tight_layout()
plt.show()


In [ ]:
port = AGENT[0].env.post_event.portfolioData
port

#### --------------------------------------

# *Multi-Agent* v2(1)

In [ ]:
from magent import MAgentThread
from master import  MasterAgentThread
from system.env import Env, SubEnv

from threading import Thread, Condition, Event, Barrier, Semaphore

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


TICKERS = ["BTC", "ETH", "SOL", "DASH"]
START = "2022-01-01"
END = "2023-10-01"
INTERVAL = "1d"
CAPITAL = 100

config = {"capital" : CAPITAL, "interval" : INTERVAL,
          "symbols" : TICKERS, "start" : START, "end" : END}
env = Env(**config)

sub_env_1 = SubEnv(symbol="BTC", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_2 = SubEnv(symbol="ETH", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_3 = SubEnv(symbol="SOL", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_4 = SubEnv(symbol="EGLD", capital=CAPITAL, interval=INTERVAL, start=START, end=END)

# Agent details
AGENT_PARAMS = [
    [(1, "BTC"), sub_env_1],
    [(2, "ETH"), sub_env_2],
    [(3, "SOL"), sub_env_3],
    [(4, "EGLD"), sub_env_4]
]

N = len(AGENT_PARAMS)

STRATEGIES = [
    ((1, "BTC"), "MOMENTUM", 3),
    ((2, "ETH"), "TRIPLEMA", (3, 7, 14)),
    ((3, "SOL"), "MOMENTUM", 8),
    ((4, "EGLD"), "TRIPLEMA", (14, 21, 35))
]

In [ ]:
master = MasterAgentThread(env)
master.set_barrier(n=N)

for agent_param, strategie in zip(AGENT_PARAMS, STRATEGIES):
    master.add_agent(*agent_param)
    master.update_params(*strategie)


master.active_agents()
master.run()

In [ ]:
master.agents

In [ ]:
#master.agents[(1, "BTC")].env.get_viz(1, "BTC")
master.agents[(AGENT_PARAMS[0][0])].env.get_viz(*AGENT_PARAMS[0][0])

In [ ]:
master.agents[(AGENT_PARAMS[1][0])].env.get_viz(*AGENT_PARAMS[1][0])

In [ ]:
master.agents[(AGENT_PARAMS[2][0])].env.get_viz(*AGENT_PARAMS[2][0])

In [ ]:
master.agents[(AGENT_PARAMS[3][0])].env.get_viz(*AGENT_PARAMS[3][0])

In [ ]:
master.agents[(4, "EGLD")].env.journal.portfolioData
#master.agents[(3, "SOL")].env.journal.tradesData

In [ ]:
master.agents

In [ ]:
master.agents[(3, "SOL")].env.journal.portfolioData

# *End*

In [ ]:
import threading

threading.enumerate()

# *Multi-Agent v3* Multiprocessing

In [ ]:
import multiprocessing
from multiprocessing import Process, Manager, Queue, Barrier, Event, Semaphore



# Plot

In [ ]:
import plotly.graph_objects as go

categories = ['processing cost','mechanical properties','chemical stability',
              'thermal stability', 'device integration']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=[1, 5, 2, 2, 3],
      theta=categories,
      fill='toself',
      name='Product A'
))
fig.add_trace(go.Scatterpolar(
      r=[4, 3, 2.5, 1, 2],
      theta=categories,
      fill='toself',
      name='Product B'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 5]
    )),
  showlegend=False
)

fig.show()

In [ ]:
class MyGenerator:
    def __init__(self, n):
        self.n = n
        self.current = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current < self.n:
            result = self.current
            self.current += 1
            return result
        else:
            raise StopIteration

# Créez une instance de la classe MyGenerator
dix_nombres = MyGenerator(10)

# Utilisez une boucle for pour obtenir les valeurs
for nombre in dix_nombres:
    print(nombre)


In [ ]:
import multiprocessing

multiprocessing.cpu_count()

In [ ]:
import os 

os.cpu_count()

In [ ]:
import threading

class Master:
    def __init__(self, condition, event):
        self.condition = condition
        self.event = event

    def run(self):
        for _ in range(10):
            with self.condition:
                print("Maître : J'attends que les agents soient prêts.")
                self.condition.wait()
                print("Maître : Autorisation d'exécuter le travail.")
                self.event.set()
                self.condition.notify_all()

class Agent:
    def __init__(self, agent_id, condition, event):
        self.agent_id = agent_id
        self.condition = condition
        self.event = event

    def run(self):
        for _ in range(10):
            with self.condition:
                print(f"Agent {self.agent_id} : Prêt à travailler.")
                self.condition.notify()
                self.condition.wait()
                self.event.wait()
                self.event.clear()
                print(f"Agent {self.agent_id} : Travail en cours.")
                self.condition.notify()
                self.condition.wait()
                print(f"Agent {self.agent_id} : Travail terminé.")

# Créez une Condition pour coordonner les agents et le maître
condition = threading.Condition()

# Créez un Event pour autoriser les agents à travailler
event = threading.Event()

# Créez le maître et les agents
master = Master(condition, event)
agents = [Agent(i, condition, event) for i in range(1, 4)]

# Créez des threads pour le maître et les agents
master_thread = threading.Thread(target=master.run)
agent_threads = [threading.Thread(target=agent.run) for agent in agents]

# Démarrez les threads
master_thread.start()
for thread in agent_threads:
    thread.start()

# Attendez que tous les threads se terminent
master_thread.join()
for thread in agent_threads:
    thread.join()

print("Tous les agents et le maître ont terminé.")


In [ ]:
import time
from threading import Thread
from queue import Queue

class Agent:
    def __init__(self, master_queue, name):
        self.master_queue = master_queue
        self.name = name

    def do_task(self, task):
        print(f"Agent {self.name} received task: {task}")
        # Simulating task execution
        time.sleep(2)
        print(f"Agent {self.name} completed task: {task}")
        # Inform the master that the task is completed
        self.master_queue.put((self.name, task))


class Master:
    def __init__(self):
        self.agent_queues = {}

    def add_agent(self, agent):
        agent_queue = Queue()
        self.agent_queues[agent.name] = agent_queue
        Thread(target=self.listen_to_agent, args=(agent, agent_queue)).start()

    def assign_task(self, agent_name, task):
        agent_queue = self.agent_queues.get(agent_name)
        if agent_queue:
            print(f"Master assigned task: {task} to Agent {agent_name}")
            agent_queue.put(task)
        else:
            print(f"Agent {agent_name} not found.")

    def listen_to_agent(self, agent, agent_queue):
        while True:
            task = agent_queue.get()
            if task is None:
                break
            print(f"Master received task completion report from Agent {agent.name} for task: {task}")


# Exemple d'utilisation
if __name__ == "__main__":
    master = Master()

    agents = [Agent(master_queue=master, name=f"Agent{i+1}") for i in range(4)]

    for agent in agents:
        master.add_agent(agent)

    master.assign_task("Agent1", "Task1")
    master.assign_task("Agent2", "Task2")
    master.assign_task("Agent3", "Task3")
    master.assign_task("Agent4", "Task4")

    # Attendez la fin des tâches avant de quitter
    time.sleep(3)  # Simuler le temps nécessaire pour terminer les tâches
    for agent in agents:
        master.add_agent(agent)  # Envoyer un signal de fin à tous les agents


In [ ]:
import multiprocessing
import time
import random

class Agent(multiprocessing.Process):
    def __init__(self, agent_id, master_queue, agent_queue):
        super().__init__()
        self.agent_id = agent_id
        self.master_queue = master_queue
        self.agent_queue = agent_queue

    def run(self):
        while True:
            instruction = self.master_queue.get()

            if instruction == "QUIT":
                break

            # Ajoutez des conditions en fonction de l'instruction
            if instruction == "Commande 1":
                result = f"Agent {self.agent_id} exécute la Commande 1 spécifique."
            elif instruction == "Commande 2":
                result = f"Agent {self.agent_id} exécute la Commande 2 spécifique."
            else:
                result = f"Agent {self.agent_id} exécute une instruction non gérée."

            self.agent_queue.put(result)

class Master:
    def __init__(self):
        self.master_queue = multiprocessing.Queue()
        self.agent_queues = [multiprocessing.Queue() for _ in range(4)]
        self.agents = [Agent(i, self.master_queue, self.agent_queues[i]) for i in range(4)]

    def start_agents(self):
        for agent in self.agents:
            agent.start()

    def send_instructions(self, instructions):
        for instruction in instructions:
            for agent_queue in self.agent_queues:
                self.master_queue.put(instruction)

            time.sleep(1)  # Attendez un certain temps pour simuler le traitement parallèle

            for _ in range(4):
                report = self.master_queue.get()
                print(report)

    def stop_agents(self):
        for _ in range(4):
            self.master_queue.put("QUIT")

        for agent in self.agents:
            agent.join()

if __name__ == "__main__":
    master_instance = Master()
    master_instance.start_agents()

    instructions = ["Commande 1", "Commande 2", "Commande 3", "Commande 4"]
    master_instance.send_instructions(instructions)

    master_instance.stop_agents()


In [ ]:
import multiprocessing
import time

class Agent(multiprocessing.Process):
    def __init__(self, agent_id, master_event, agent_queue):
        super().__init__()
        self.agent_id = agent_id
        self.master_event = master_event
        self.agent_queue = agent_queue

    def run(self):
        while True:
            # Wait for the event to be set by the master
            self.master_event.wait()

            instruction = self.agent_queue.get()

            if instruction == "QUIT":
                break

            # Your conditions or logic here
            result = f"Agent {self.agent_id} a exécuté l'instruction: {instruction}"

            # Report the result to the master
            self.agent_queue.put(result)

class Master:
    def __init__(self):
        self.master_event = multiprocessing.Event()
        self.agent_queues = [multiprocessing.Queue() for _ in range(4)]
        self.agents = [Agent(i, self.master_event, self.agent_queues[i]) for i in range(4)]

    def start_agents(self):
        for agent in self.agents:
            agent.start()

    def send_instructions(self, instructions):
        for instruction in instructions:
            # Send the instruction to all agents
            for agent_queue in self.agent_queues:
                agent_queue.put(instruction)

            # Set the event to signal agents to start processing
            self.master_event.set()

            time.sleep(1)  # Simulate parallel processing time

            # Clear the event for the next round
            self.master_event.clear()

            # Collect reports from agents
            for _ in range(4):
                report = self.agent_queues[0].get()  # Assuming all agents provide the same result
                print(report)

    def stop_agents(self):
        # Send the QUIT instruction to all agents
        for agent_queue in self.agent_queues:
            agent_queue.put("QUIT")

        # Wait for agents to finish
        for agent in self.agents:
            agent.join()

if __name__ == "__main__":
    master_instance = Master()
    master_instance.start_agents()

    instructions = ["Commande 1", "Commande 2", "Commande 3", "Commande 4"]
    master_instance.send_instructions(instructions)

    master_instance.stop_agents()


In [ ]:


class ProfitManagement:
    def __init__(self, profit_target_percentage, trailing_stop_percentage):
        self.profit_target_percentage = profit_target_percentage
        self.trailing_stop_percentage = trailing_stop_percentage
        self.entry_price = None
        self.profit_target_price = None
        self.trailing_stop_price = None

    def set_entry_price(self, entry_price):
        self.entry_price = entry_price
        self.profit_target_price = entry_price * (1 + self.profit_target_percentage)
        self.trailing_stop_price = entry_price * (1 - self.trailing_stop_percentage)

    def take_profit(self, current_price):
        if current_price >= self.profit_target_price:
            return True
        return False

    def trailing_stop_loss(self, current_price):
        if current_price >= self.trailing_stop_price:
            self.trailing_stop_price = current_price * (1 - self.trailing_stop_percentage)
        return self.trailing_stop_price

# Exemple d'utilisation
profit_target_percentage = 0.05  # Objectif de profit de 5%
trailing_stop_percentage = 0.02  # Trailing stop-loss de 2%

profit_manager = ProfitManagement(profit_target_percentage, trailing_stop_percentage)
profit_manager.set_entry_price(100)  # Prix d'entrée de la position

# Simulation du mouvement des prix
price_history = [100, 110, 120, 115, 125, 130, 135, 130, 140, 145]

for price in price_history:
    if profit_manager.take_profit(price):
        print(f"Prendre profit au prix {price}")
        break
    else:
        trailing_stop_price = profit_manager.trailing_stop_loss(price)
        print(f"Prix actuel : {price}, Trailing stop-loss : {trailing_stop_price}")
